In [1]:
# external api
import fundamentalanalysis as fa
import yfinance as yf

# data analytic 
import numpy as np
import pandas as pd

# additional liberies
import datetime
import statistics as st
import itertools

# ML strategies
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# ML analyse
from sklearn import preprocessing
from sklearn.decomposition import PCA

# vizualization
import pylab as plb
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a'  #job.chap@icloud.com
# api_key = 'c81352430e2fe3c941faf0814227562b'  #jobs.chaps@gmail.com
api_key = 'a765d11740cccfb61177da0ad8699d1e'  #job.chap@gmail.com

In [4]:
''' Тикеры '''
tick = 'F'  # поменял название переменно ticker на tick, для настройки по одной акции
tickers_list = ['F', 'AAPL' ,'ADBE']
                # 'COP', 'AMAT', 'MFGP', 'FISV', 'ASAN', 'ALIT', 'DAVA', 'NCR', 'SMTC', 'EXLS', 'JKHY', 'ORCL', 'APPS', 'U', 'FICO', 'WEX', 'TXN', 'GDDY', 'CRSR', 'MANH', 'LITE', 'MANH', 'TENB', 'ACLS', 'LPL', 'GWRE', 'PAYC', 'NCNO', 'APH', 'AVT', 'COUP']

                
''' для фильтрации'''
# filt_start_date = '2000'
# filt_finish_date = '2022'


''' время '''
stime = '2000-01-01'
ftime = '2023-02-01'
period = '1mo'  # '1d' '1wk'

Класс Fund ticker

In [5]:
class Fund_ticker:
    def __init__(self, tick, stime, ftime, api_key_fund):
        self.tick = tick
        self.stime = stime
        self.ftime = ftime
        self.api_key_fund = api_key_fund
        
        
    def _change_index_plus_one(self, df):
        """ добавлят цифру к году в фундаментальные api, чтобы видень данные как начало года, а не как факт 
        """
        
        df_index = df.index.to_list()
        
        for i in range(len(df_index)):
            df_index[i] = str(int(df_index[i])+1)
            
        df['Date_index'] = df_index
        
        df = df.set_index('Date_index')
        
        return df
        
    def get_key_metr(self):
        """ получаем df с key_metrics
        """
        
        df_key_metrics = fa.key_metrics(self.tick, self.api_key_fund, period='annual')
        df_key_metrics = self._change_index_plus_one(df_key_metrics.T)
        
        return df_key_metrics
    
    def get_income_statement(self):
        """ получаем df с income_statement
        """   
        
        income_statement = fa.income_statement(self.tick, self.api_key_fund, period='annual')
        income_statement = self._change_index_plus_one(income_statement.T)
        
        return income_statement
        
    
    def get_balance_sheet_statement(self):
        """ получаем df с balance_sheet_statement
        """  
        
        balance_sheet_statement = fa.balance_sheet_statement(self.tick, self.api_key_fund, period='annual')
        balance_sheet_statement = self._change_index_plus_one(balance_sheet_statement.T)
        
        return balance_sheet_statement

    def get_cash_flow_statement(self):   
        """ получаем df с cash_flow_statement
        """    
        
        cash_flow_statement = fa.cash_flow_statement(self.tick, self.api_key_fund, period='annual')
        cash_flow_statement = self._change_index_plus_one(cash_flow_statement.T)
        
        return cash_flow_statement

    def get_financial_ratios(self):    
        """ получаем df с financial_ratios
        """
        
        financial_ratios = fa.financial_ratios(self.tick, self.api_key_fund, period='annual')
        financial_ratios = self._change_index_plus_one(financial_ratios.T)
        
        return financial_ratios

    def get_financial_statement_growth(self):
        """ получаем df с financial_statement_growth
        """ 
        
        financial_statement_growth = fa.financial_statement_growth(self.tick, self.api_key_fund, period='annual')
        financial_statement_growth = self._change_index_plus_one(financial_statement_growth.T)
        
        return financial_statement_growth
    
    
    def all_metrics(self):
        """ df со всеми колонками = конкатенация колонок со всех полученных таблиц со значениями
        """
        
        df_km = self.get_key_metr()
        df_is = self.get_income_statement()
        df_bss = self.get_balance_sheet_statement()
        df_cfs = self.get_cash_flow_statement()
        df_fr = self.get_financial_ratios()
        df_fsg = self.get_financial_statement_growth()
        
        df_all_fa_columns = pd.concat([df_km,
                                       df_is,
                                       df_bss,
                                       df_cfs,
                                       df_fr,
                                       df_fsg
                                       ], axis=1)
        
        """ убраем дубликаты колонок """
        df_all_fa_columns = df_all_fa_columns.loc[:,~pd.concat([df_km,
                                                                df_is,
                                                                df_bss,
                                                                df_cfs,
                                                                df_fr,
                                                                df_fsg
                                                                ], axis=1).columns.duplicated()]
        
        
        """ удаляются не используемые колонки """
        df_all_fa_columns = df_all_fa_columns.drop(['period', 'reportedCurrency', 'cik', 'fillingDate', 'calendarYear', 'link', 'finalLink'], axis=1)        
        
        return df_all_fa_columns
    
    def all_metrics_period(self):
        """ df со всеми колонками отфильтрованная в нужный период
        """
        
        df = self.all_metrics()
        
        # берем из дат 4е символа
        start = self.stime[:4]
        finish = self.ftime[:4]
        
        df_filt = df.loc[finish:start].copy(deep=True)
        
        return df_filt  
        
    def get_yahoo_prices(self, ticker):
        ''' получаем цены из yahoo_finance и меняем методом индекс
        '''
        ticker = yf.Ticker(ticker)

        df = ticker.history(start=self.stime, end=self.ftime, interval='1mo')
        x = pd.DataFrame(df)
        x.rename(columns={"Close": self.tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        
        return z
    
    def _change_index_yahoo(self, df):
        ''' сокращаем индекс из dataframe yahoo
        '''
        

        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df  
    
    def _get_years_prices(self, df, list_years):
        ''' из датафрейма цен получает цены на начало года
        '''
        
        # df = all_metrics_period()
        # index_year = df.index.to_list()
        
        year_prices = []
        for i in range(len(list_years)):
            x = int(list_years[i])
            x = str(x)
            r = df.loc[f'{x}-01-01']
            r = float(r)
            year_prices.append(r)

        return year_prices
    
    
    def df_with_prices(self):
        
        df = self.all_metrics_period()
        
        df_ticker = self._change_index_yahoo(self.get_yahoo_prices(self.tick))  # ценs нужной акции + измененный индекс
        df_snp = self._change_index_yahoo(self.get_yahoo_prices('^GSPC'))  # цены SNP + измененный индекс       
        
        index_years = df.index.to_list()
        
        df_ticker_years = self._get_years_prices(df_ticker, index_years)
        df_snp_years = self._get_years_prices(df_snp, index_years)
        
        
        df['ticker'] = self.tick
        df['stock_price'] = df_ticker_years
        df['SNP_price'] = df_snp_years
        
        return df
        
        
    def _change_percent_all_columns(self):
        ''' получаем относительные значения, с исключениями:
        а) если не получается обработать вообще - пропускает
        б) если уходит ниже нуля или выше, то назначает значение 'Minus' или 'Plus'
        '''
        df = self.df_with_prices()
        x = df.columns.to_list()
        
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        if v[y] < 0 and v[y+1] > 0:
                            l.append('Minus')
                        elif v[y] > 0 and v[y+1] < 0:
                            l.append('Plus')
                        else:
                            vv = (v[y] / (v[y+1] / 100)) - 100
                            l.append(vv)  
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)

            df_copy = df.copy() 
            df_copy[f'change % {x[i]}'] = l
            df = pd.concat([df, df_copy[f'change % {x[i]}']], axis=1)
            
        df = df.drop(['change % acceptedDate','change % ticker'], axis=1)
        
        """ удаляем последнюю строку, где значения равны 0"""
        df = df.drop(df.index[-1])
            
        return df
        
    
    def df_add_two_next_percent_years(self):
        ''' добавлет два следующих года для акции и для рынка
        + если нет, то добавляет None
        '''
        
        df = self._change_percent_all_columns()
        
        # первый год акция
        stock_prices = df['change % stock_price'].to_list()
        stock_prices_first = ['None'] + stock_prices
        stock_prices_first.pop()

        # второй год акция
        stock_prices_second = ['None'] + stock_prices_first
        stock_prices_second.pop()

        # первый год рынок
        snp_prices = df['change % stock_price'].to_list()
        snp_prices_first = ['None'] + snp_prices
        snp_prices_first.pop()

        # второй год рынок
        snp_prices_second = ['None'] + snp_prices_first
        snp_prices_second.pop()

        df_copy = df.copy()
        df_copy['stock_plus_1year'] = stock_prices_first
        df_copy['stock_plus_2year'] = stock_prices_second
        df_copy['snp_plus_1year'] = snp_prices_first
        df_copy['snp_plus_2year'] = snp_prices_second

        return df_copy    

    
    def df_all_with_category(self):
        ''' добавляем категоричные значение
        ''' 
        
        df = self.df_add_two_next_percent_years()
        
        t = []
        for i in range(len(df['stock_plus_1year'])):
            try:
                if df['stock_plus_1year'][i] > 0:
                    t.append(1)
                else:
                    t.append(0)
            except:
                t.append('None')

        df['categor_1year'] = t        

        t = []
        for i in range(len(df['stock_plus_2year'])):
            try:
                if df['stock_plus_2year'][i] > 0:
                    t.append(1)
                else:
                    t.append(0)
            except:
                t.append('None')

        df['categor_2year'] = t    
        
        return df

Класс Prepare_data

In [6]:
class Prepare_data:
    def __init__(self, tickers_list, stime, ftime, api_key):
        self.ticker_list = tickers_list
        self.stime = stime
        self.ftime = ftime
        self.api_key = api_key
        self.df = Fund_ticker(tickers_list[0], stime, ftime, api_key).df_all_with_category() # сразу получаем 
        
        
    def change_names_for_combimation(self, df):  # в процессе 
        """ получаем колонки которыем мы будем использовать для создания комбинаций для ml
        Требования: 
        - содержат 'change %' - для анализа по отновсительным значениям
        - только данные формата int и float (исключаем Nan, Minus, Plus и прочее) - чтобы использовать больше моделей ML и более точно
        """
        
        df_new = df
        
        list_df_columns = df_new.columns.to_list()
        

        t = []
        for i in range(len(list_df_columns)):
            if 'change %' in list_df_columns[i]  and df[f'{list_df_columns[i]}'].dtype == float: 
                t.append(list_df_columns[i])
            else:
                pass
     
        return t
    
    def compination_change_names(self, df):
        """ получаем комбинации возможных значений
        """
        
        names_list = self.change_names_for_combimation(df)
        combinations = list(itertools.combinations(names_list , 2))
        
        return combinations
    
    def split_combinations(self, df):
        """ разбиваем комбинации на более короткие (по 100 штук)
        """
                
        values_combo = self.compination_change_names(df)
        split_lists = [values_combo[i:i+100] for i in range(0, len(values_combo), 100)]
             
        return split_lists
    
    
    def concat_tickers_tables(self):
        """ загружаем список тикеров, получаем обработанную таблицу с объединенными тикерами
        """
    
        tick_list = self.ticker_list
    
        
        try:
            for i in range(len(tick_list)):
                if i == 0:
                    df = Fund_ticker(tick_list[i], self.stime, self.ftime, self.api_key).df_all_with_category()
                    print(tick_list[i])
                else:
                    try:
                        df = pd.concat([df, Fund_ticker(tick_list[i], self.stime, self.ftime, self.api_key).df_all_with_category()], ignore_index=True)
                        print(tick_list[i])
                    except:
                        print(f'{tick_list[i]} = не прошел')
        except:
            df = 0

        return df
    
    
    def df_clean_changes(self):
    
        """ очищаем строки с None df по колонке "categor_1year" если целевые величины будут браться по ней
        """
        
        df = self.concat_tickers_tables()
        
        
        # выбираем колонки, содержащие слово "нет" в значениях
        cols_with_word = df.applymap(lambda x: 'Plus' in str(x)).any()

        # удаляем выбранные колонки
        df.drop(cols_with_word[cols_with_word == True].index, axis=1, inplace=True)


        cols_with_word = df.applymap(lambda x: 'Minus' in str(x)).any()


        # удаляем выбранные колонки
        df.drop(cols_with_word[cols_with_word == True].index, axis=1, inplace=True)
        
        return df
    
    
    def df_clean_first_year_ct(self):
        """ очищаем строки с None df по колонке "categor_1year" если целевые величины будут браться по ней
        """
        
        df = self.df_clean_changes()
        
        df = df.drop(df[df['categor_1year'] == 'None'].index)
        
        return df
    
    
    def df_clean_second_year_ct(self):
        """ очищаем строки с None df по колонке "categor_1year" если целевые величины будут браться по ней
        """
        
        df = self.df_clean_changes()
        
        df = df.drop(df[df['categor_2year'] == 'None'].index)
        
        return df

ML_analyse

In [7]:
class PCA_analyse:
    """ модели для анализа таблицы
    """
    
    def __init__(self, tickers_list, stime, ftime, api_key):
        self.tickers_list = tickers_list
        self.stime = stime
        self.ftime = ftime
        self.api_key = api_key
        
    def pca_general_comp(self):
        """ анализ главных компонентов Principal Components Analysis
        """
        
        
        df = Prepare_data(self.tickers_list, self.stime, self.ftime, self.api_key).df_clean_second_year_ct()
        change_names = Prepare_data(self.tickers_list, self.stime, self.ftime, self.api_key).change_names_for_combimation(df)
        
        X = df[change_names+["stock_plus_1year", "stock_plus_2year", "snp_plus_1year", "snp_plus_2year"]]
        y = df["categor_2year"]
        
        X = preprocessing.SplineTransformer().fit(X).transform(X)
        
        model = PCA()
        results = model.fit(X)
        Z = results.transform(X)
        
        return results
    
    def vizualization_general_comp_pca(self):
        """ визуализируем итог главных компонентов PCA
        """
        
        results = self.pca_general_comp()
        plb.plot(results.explained_variance_)
        plb.show()
        

    def table_general_comp_pca(self):
        """ получаем таблицу итога главных компонентов PCA
        """
        
        results = self.pca_general_comp()
        
        y = results.explained_variance_

        df = pd.DataFrame(np.round(y, decimals=4), columns=['Influence of components'])
        
        return df
    
    def nbg_without_pca(self):
        
        df = Prepare_data(self.tickers_list, self.stime, self.ftime, self.api_key).df_clean_second_year_ct()
        change_names = Prepare_data(self.tickers_list, self.stime, self.ftime, self.api_key).change_names_for_combimation(df)
        
        X = df[change_names+["stock_plus_1year", "stock_plus_2year", "snp_plus_1year", "snp_plus_2year"]]
        y = df["categor_2year"]
        
        X = preprocessing.SplineTransformer().fit(X).transform(X)
        y = preprocessing.LabelEncoder().fit_transform(y)
        
        
        gnb = GaussianNB()
        fit = gnb.fit(X,y)
        pred = fit.predict(X)
        
        return confusion_matrix(pred, y)
    
    
    def pca_nbg(self):
        """ 
        """

In [8]:
# x = PCA_analyse(tickers_list, stime, ftime, api_key).pca_general_comp()

In [9]:
x = PCA_analyse(tickers_list, stime, ftime, api_key).nbg_without_pca()

F
AAPL
ADBE


In [10]:
# x = PCA_analyse(tickers_list, stime, ftime, api_key).table_general_comp_pca()

In [14]:
x

array([[21,  0],
       [ 0, 42]], dtype=int64)